# Import Library

In [73]:
import pandas as pd
import os
import openpyxl
from openpyxl.utils import get_column_letter, column_index_from_string
import time
import locale
from babel.dates import format_date



In [74]:
start_time = time.time()

# Data Cleaning

## Data

### Path

In [75]:
current_dir = os.getcwd()
current_dir

'/Users/arsenoferialzahabi/Library/CloudStorage/OneDrive-Personal/Personal/BEI/IDXTION 2'

In [76]:
def load_data_folder(nama_folder):
    data = pd.DataFrame()

    for i in os.listdir(nama_folder):
        data_temp = pd.read_excel(os.path.join(current_dir,nama_folder, i),header=1)
        header_unnamed = data_temp.columns.str.contains('Unnamed')
        data_temp = data_temp.loc[:, ~header_unnamed]
        data_temp['Company'] = nama_folder
        data_temp['Date'] = pd.to_datetime(i.replace('.xlsx',''))
        data_temp['Date'] = data_temp['Date'].dt.date
        data = pd.concat([data, data_temp], ignore_index=True)

    return data

### Account Mapping

In [77]:
def load_account_mapping():
    data = pd.read_excel('Account Mapping.xlsx',header=2)
    data = data.drop(columns=['Unnamed: 4'])
    return data

load_account_mapping()

,Header,FSLI,LV 1,LV 2
0,Aset Lancar,Kas Setara kas,10.110.0000,10.111.1000
1,Aset Lancar,Kas Setara kas,10.110.0000,10.111.2101
2,Aset Lancar,Kas Setara kas,10.110.0000,10.111.2102
3,Aset Lancar,Kas Setara kas,10.110.0000,10.111.2103
4,Aset Lancar,Kas Setara kas,10.110.0000,10.111.2104
...,...,...,...,...
401,Pajak Final,Pajak Final,60.100.0002,40.510.3000
402,NaN,NaN,NaN,NaN
403,Beban Pajak,Beban Pajak,60.200.0001,60.200.0000
404,Beban Pajak,Beban Pajak,60.200.0001,60.300.0000


### CONSOL

In [78]:
def consol_data(company_list: list):
    data = pd.DataFrame()
    for company in company_list:
        data_temp = load_data_folder(company)
        data = pd.concat([data, data_temp], ignore_index=True)
    return data

### CONSOL FULL

In [79]:
def consol_full_data(company_list: list):
    data_consol = consol_data(company_list)
    account_mapping = load_account_mapping().set_index('LV 2')
    data_full_consol = data_consol.join(account_mapping, on='Coa Grouping Lv 2', how='left')
    return data_full_consol



In [80]:
consol_full_data(['BEI','PHEI','SIPF'])['Date'].unique()
consol_data=consol_full_data(['BEI','PHEI','SIPF'])

## Export CSV

In [81]:
consol_data.to_csv(os.path.join(current_dir,'Clean','consol_full_data.csv'), index=False)

# Consol Report

### Data

In [82]:
consol_data['Date'] = pd.to_datetime(consol_data['Date'])
consol_data['FSLI'] = consol_data['FSLI'].str.lower().str.replace(' ','_').str.replace(',', '').str.replace('-','_').str.strip()
current_year = consol_data['Date'].max().year
previous_year = current_year - 1

## Current Year Data
current_year_data = consol_data[consol_data['Date'].dt.year == current_year]
current_year_data = current_year_data[current_year_data['Date']==current_year_data['Date'].max()]
current_year_data['FSLI'] = current_year_data['FSLI']+"_1"

## Previous Year Data
previous_year_data = consol_data[consol_data['Date'].dt.year == previous_year]
previous_year_data = previous_year_data[previous_year_data['Date']==previous_year_data['Date'].max()]
previous_year_data['FSLI'] = previous_year_data['FSLI']+"_0"

consol_data

,NO COA,COA NAME,Balance,Coa Grouping Lv 2,Company,Date,Header,FSLI,LV 1
0,1010101,PETTY CASH,0.000000e+00,10.111.1000,BEI,2024-12-31,Aset Lancar,kas_setara_kas,10.110.0000
1,1010102,BANK BNI 824 RP.,2.142290e+09,10.111.2101,BEI,2024-12-31,Aset Lancar,kas_setara_kas,10.110.0000
2,1010103,BANK BNI 846 RP.,1.867835e+08,10.111.2101,BEI,2024-12-31,Aset Lancar,kas_setara_kas,10.110.0000
3,1010104,BANK MANDIRI 573 RP,0.000000e+00,10.111.2103,BEI,2024-12-31,Aset Lancar,kas_setara_kas,10.110.0000
4,1010105,BANK MANDIRI 858 RP.,0.000000e+00,10.111.2103,BEI,2024-12-31,Aset Lancar,kas_setara_kas,10.110.0000
...,...,...,...,...,...,...,...,...,...
3530,55241.0,Pembahasan Peraturan/ SE OJK,8.708600e+07,50.300.6000,SIPF,2022-12-31,Beban,pengembangan_pasar_modal,50.300.0000
3531,11431.0,Piutang Bunga EBA - Operasional,3.227671e+07,10.150.2200,SIPF,2022-12-31,Aset Lancar,piutang_lain_lain_neto,10.150.0000
3532,61140.0,Bunga EBA,-3.227671e+07,40.510.2000,SIPF,2022-12-31,Pendapatan Lainnya,pendapatan_investasi,40.600.4002
3532,61140.0,Bunga EBA,-3.227671e+07,40.510.2000,SIPF,2022-12-31,Pajak Final,pajak_final,60.100.0002


### Path

In [83]:
report_path = os.path.join(os.getcwd(),"Report Consol")
template_path = os.path.join(os.getcwd(),"Report Template","Template.xlsx")

### Open File

In [84]:
wb = openpyxl.load_workbook(template_path)

In [85]:
def fill_name_manager(name_value, value):
    try:
        definition = wb.defined_names[name_value]
        for sheet_name, coord in definition.destinations:
            clean_coord = coord.replace('$','')
            wb[sheet_name][clean_coord] = value
    except:
        print(f"Defined name {name_value} not found in the workbook.")

### Aset Lancar

In [86]:
asset_lancar_current = current_year_data[current_year_data['Header']=='Aset Lancar']
asset_lancar_current = asset_lancar_current.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
asset_lancar_current

Date,2024-12-31
FSLI,
aset_keuangan_lainnya_1,2.742702e+12
deposito_rupiah_1,5.782000e+11
deposito_usd_1,5.461066e+10
kas_setara_kas_1,6.795334e+09
ksa_dan_setara_kas_yang_dibatas_penggunaanya_1,4.039838e+10
pajak_dibayar_dimuka_1,1.983519e+10
piutang_lain_lain_neto_1,7.918320e+10
piutang_usaha_neto_1,5.031132e+11
uang_muka_dan_biaya_dibayar_dimuka___bagian_lancar_1,4.250660e+10


In [87]:
for index, row in asset_lancar_current.iterrows():
    fill_name_manager(index, row.values[0])



Defined name deposito_rupiah_1 not found in the workbook.
Defined name deposito_usd_1 not found in the workbook.


In [88]:
aset_lancar_previous = previous_year_data[previous_year_data['Header']=='Aset Lancar']
aset_lancar_previous = aset_lancar_previous.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
aset_lancar_previous

Date,2023-12-31
FSLI,
aset_keuangan_lainnya_0,2.222969e+12
deposito_rupiah_0,8.562000e+11
deposito_usd_0,4.631700e+10
kas_setara_kas_0,7.003334e+09
pajak_dibayar_dimuka_0,6.070080e+08
piutang_lain_lain_neto_0,7.546102e+10
piutang_usaha_neto_0,4.885530e+11
uang_muka_dan_biaya_dibayar_dimuka___bagian_lancar_0,3.490308e+10


In [89]:
for index, row in aset_lancar_previous.iterrows():
    fill_name_manager(index, row.values[0])

Defined name deposito_rupiah_0 not found in the workbook.
Defined name deposito_usd_0 not found in the workbook.


### Tanggal

In [90]:
current_date = consol_data[consol_data['Date'].dt.year == current_year]['Date'].max()
previous_date = consol_data[consol_data['Date'].dt.year == previous_year]['Date'].max()

# Current month bahasa indonesia contoh desember - 31
en_current_month_name = current_date.strftime('%B %d')
en_current_year = current_date.strftime('%Y')
id_current_year = format_date(current_date, format='yyyy', locale='id_ID')
id_current_month_name = format_date(current_date, format='LLLL dd', locale='id_ID')

# Previous Month Year
en_previous_month_name = previous_date.strftime('%B %d')
en_previous_year = previous_date.strftime('%Y')
id_previous_year = format_date(previous_date, format='yyyy', locale='id_ID')
id_previous_month_name = format_date(previous_date, format='LLLL dd', locale='id_ID')

#inserrt to IS
fill_name_manager('id_current_month', id_current_month_name)
fill_name_manager('en_current_month', en_current_month_name)
fill_name_manager('en_current_year', en_current_year)
fill_name_manager('en_previous_year', en_previous_year)
fill_name_manager('id_previous_month', id_previous_month_name)
fill_name_manager('en_previous_year', en_previous_year)


### Aset Tidak Lancar

In [91]:
aset_tidak_lancar_current = current_year_data[current_year_data['Header']=='Aset Tidak Lancar']
aset_tidak_lancar_current = aset_tidak_lancar_current.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
aset_tidak_lancar_current

Date,2024-12-31
FSLI,
aset_hak_guna_1,1.525073e+11
aset_lain_lain_1,2.237575e+09
aset_pajak_tangguhan_1,5.930586e+10
aset_tetap_neto_1,5.481237e+11
dana_cadangan_ganti_rugi_pemodal_1,1.536076e+11
investasi___tidak_lancar_1,6.603307e+09
investasi_pada_entitas_asosiasi_1,2.112886e+12
lebih_bayar_pajak_penghasilan_1,0.000000e+00
piutang_lain_lain_1,0.000000e+00


In [92]:
for index, row in aset_tidak_lancar_current.iterrows():
    fill_name_manager(index, row.values[0])

Defined name lebih_bayar_pajak_penghasilan_1 not found in the workbook.


In [93]:
aset_tidak_lancar_previous = previous_year_data[previous_year_data['Header']=='Aset Tidak Lancar']
aset_tidak_lancar_previous = aset_tidak_lancar_previous.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
aset_tidak_lancar_previous

Date,2023-12-31
FSLI,
aset_hak_guna_0,5.032923e+10
aset_lain_lain_0,2.287525e+09
aset_pajak_tangguhan_0,5.039325e+10
aset_tetap_neto_0,4.535318e+11
dana_cadangan_ganti_rugi_pemodal_0,1.573631e+11
investasi_pada_entitas_asosiasi_0,2.106286e+12
piutang_lain_lain_0,0.000000e+00
uang_muka_dan_biaya_dibayar_dimuka___setelah_dikurangi_bagian_lancar_0,0.000000e+00


In [94]:
for index, row in aset_tidak_lancar_previous.iterrows():
    fill_name_manager(index, row.values[0])

### Liabilitas Jangka Pendek

In [95]:
liabilitas_jangka_pendek_current = current_year_data[current_year_data['Header']=='Liabilitas Jangka Pendek']
liabilitas_jangka_pendek_current = liabilitas_jangka_pendek_current.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
liabilitas_jangka_pendek_current.iloc[:,0] = liabilitas_jangka_pendek_current.iloc[:,0]*-1
liabilitas_jangka_pendek_current

Date,2024-12-31
FSLI,
beban_akrual_1,2.728364e+11
liabilitas_sewa___bagian_jangka_pendek_1,-0.000000e+00
pendapatan_diterima_dimuka___1,2.191377e+09
utang_jasa_transaksi_1,1.034308e+11
utang_lain_lain_1,2.532003e+10
utang_pajak__1,3.103883e+11


In [96]:
for index, row in liabilitas_jangka_pendek_current.iterrows():
    fill_name_manager(index, row.values[0])

In [97]:
liabilitas_jangka_pendek_previous = previous_year_data[previous_year_data['Header']=='Liabilitas Jangka Pendek']
liabilitas_jangka_pendek_previous = liabilitas_jangka_pendek_previous.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
liabilitas_jangka_pendek_previous.iloc[:,0] = liabilitas_jangka_pendek_previous.iloc[:,0]*-1
liabilitas_jangka_pendek_previous

Date,2023-12-31
FSLI,
beban_akrual_0,1.734126e+11
pendapatan_diterima_dimuka___0,1.773730e+09
utang_jasa_transaksi_0,1.058287e+11
utang_lain_lain_0,1.486629e+10
utang_pajak__0,3.202625e+11


In [98]:
for index, row in liabilitas_jangka_pendek_previous.iterrows():
    fill_name_manager(index, row.values[0])

### Liabilitas Jangka Panjang

In [99]:
liabilitas_jangka_panjang_current = current_year_data[current_year_data['Header']=='Liabilitas Jangka Panjang']
liabilitas_jangka_panjang_current = liabilitas_jangka_panjang_current.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
liabilitas_jangka_panjang_current.iloc[:,0] = liabilitas_jangka_panjang_current.iloc[:,0]*-1
liabilitas_jangka_panjang_current

Date,2024-12-31
FSLI,
liabilitas_imbalan_kerja_1,1.057175e+11
liabilitas_sewa___bagian_jangka_panjang_1,1.696976e+11


In [100]:
for index, row in liabilitas_jangka_panjang_current.iterrows():
    fill_name_manager(index, row.values[0])

In [101]:
liabilitas_jangka_panjang_previous = previous_year_data[previous_year_data['Header']=='Liabilitas Jangka Panjang']
liabilitas_jangka_panjang_previous = liabilitas_jangka_panjang_previous.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
liabilitas_jangka_panjang_previous.iloc[:,0] = liabilitas_jangka_panjang_previous.iloc[:,0]*-1
liabilitas_jangka_panjang_previous

Date,2023-12-31
FSLI,
liabilitas_imbalan_kerja_0,9.350056e+10
liabilitas_sewa___bagian_jangka_panjang_0,5.498585e+10


In [102]:
for index, row in liabilitas_jangka_panjang_previous.iterrows():
    fill_name_manager(index, row.values[0])

### Ekuitas

In [103]:
ekuitas_current = current_year_data[current_year_data['Header']=='Ekuitas']
ekuitas_current = ekuitas_current.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
ekuitas_current.iloc[:,0] = ekuitas_current.iloc[:,0]*-1
ekuitas_current

Date,2024-12-31
FSLI,
agio_saham_1,8.272100e+09
modal_saham___nilai_nominal_1,1.027500e+12
penghasilan_komprehensif_lain_1,-3.783618e+09
saham_diperoleh_kembali_1,-6.000000e+10
saldo_laba_1,4.762923e+12


In [104]:
for index,row in ekuitas_current.iterrows():
    fill_name_manager(index, row.values[0])

In [105]:
ekuitas_previous = previous_year_data[previous_year_data['Header']=='Ekuitas']
ekuitas_previous = ekuitas_previous.pivot_table(index='FSLI', columns='Date', values='Balance', aggfunc='sum')
ekuitas_previous.iloc[:,0] = ekuitas_previous.iloc[:,0]*-1
ekuitas_previous

Date,2023-12-31
FSLI,
agio_saham_0,6.215000e+09
modal_saham___nilai_nominal_0,1.027500e+12
penghasilan_komprehensif_lain_0,1.927806e+10
saham_diperoleh_kembali_0,-3.000000e+10
saldo_laba_0,4.449041e+12


In [106]:
for index,row in ekuitas_previous.iterrows():
    fill_name_manager(index, row.values[0])

### Close File

In [107]:
wb.save(os.path.join(report_path,'Consol.xlsx'))
wb.close()

# Time consume

In [108]:
end_time = time.time()
runtime = end_time - start_time

print(f"Pembuatan Consolidation Report Membutuhkan waktu {runtime:.6f} detik")

Pembuatan Consolidation Report Membutuhkan waktu 1.009415 detik
